# Background

# Requirements

1. [Spectre CCE code](https://spectre-code.org/tutorial_cce.html)

2. [BMS trasnformation code (scri)](https://scri.readthedocs.io/en/latest/README.html)

3. python packages: numpy, h5py, scipy


# How to compute CCE strains

## Spectre CCE format

1. We first convert the boundary condition data, saved in binray format, into a format readable by the `Spectre CCE`. This is done by the `./athk_to_spectre.py` python script. Before lunching the script, we need to know the `dump_extraction_radius`, which can be find in the `AthenaK` parameter file. We usually set the `dump_extraction_radius` parameter at the middle of inner and outer radii of the shell where the metric data are interplated.

```bash

# converting binary dump data into spectre reable format:
./athk_to_spectre.py -ftype bin -radius [dump_extraction_radius] -d_out [output_directory]

```

2. Having run the convertor script, we see inside the output directory there is a `h5` file readable by `Spectre CCE`. The file name is similar to `CceR0XXX.h5`. We specify the path of this file inside the `Spectre CCE` yaml parameter file `CharacteristicExtract.yaml`; namely, we set the paramter `BoundaryDataFilename = path/to/CceR0XXX.h5` inside `CharacteristicExtract.yaml`. Then we lunch the `Spectre CCE` as follows:

```bash
./CharacteristicExtract --input-file CharacteristicExtract.yaml

```


# Debugging tools